<a href="https://colab.research.google.com/github/anaiy2004/SafePaths/blob/main/DataPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#scraping data from website
!pip install -q tabula-py
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
pdfList = list()
url = 'https://ucpd.berkeley.edu/alerts-data/daily-crime-log'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')
for link in soup.find_all('a'):
    pdfList.append((link.get('href')))
pdfList = [element for element in pdfList if len(element) > 3 and element[-4:] == '.pdf'][1:]

In [ ]:
#reading pdfs from scraped links
from tabula import read_pdf
from tabulate import tabulate
import io
pages = 3
counter = -1
ans = []
for newUrl in pdfList:
  counter += 1
  for k in range(10):
    print(counter)
  for i in range(pages):
    try:
      pdf = read_pdf(newUrl,pages = pages - i, multiple_tables = False, stream = True)
    except:
      print("An exception occurred")
  table = tabulate(pdf)
  try:
    df = pd.read_fwf(io.StringIO(table))
  except:
    continue
  df  = df.iloc[: , :6]
  df.columns = ["case", "crime", "reported", "range", "location", "disposition"]
  ans.append(df)

In [ ]:
#creating dates list
dates = []
for dataframes in ans:
  reported = list(dataframes['reported'])
  for date in reported:
    if(not ("NaN" in date)):
      dates.append(date)
      break

In [ ]:
len(ans)

164

In [ ]:
#parsing tables to find respective location and crimes
finalLocations = [[]]
finalCrimes = [[]]
counter = -1
for j in range(len(ans)):
  counter += 1
  locColumn = ans[j]['location']
  crimeColumn = ans[j]['crime']
  loc = list()
  crime = list()
  check = True
  counter2 = -1
  for strings in locColumn:
    counter2 += 1
    if(not isinstance(strings, str)):
      finalLocations.append([])
      finalCrimes.append([])
      check = False
      break
    if(not "NaN" in strings):
      loc.append(strings)
      ans1 = str(crimeColumn[counter2])
      for k in range(counter2 + 1, len(locColumn)):
        if("NaN" in locColumn[k]):
          ans1 += str(crimeColumn[k])
        else:
          break
      crime.append(ans1)
  if(not check):
    continue
  loc = loc[0:-1]
  crime = crime[0:-1]
  for i in range(len(loc) - 1):
      loc[i] = (loc[i][2:].strip())
  loc[-1] = loc[-1][6: -15]
  loc = [loc[k].split(";")[0] for k in range(len(loc))]
  finalLocations.append(loc)
  finalCrimes.append(crime)
finalLocations = finalLocations[1:]
finalCrimes = finalCrimes[1:]

In [ ]:
finalLocations[0]

['1928 Stadium Rim Wy',
 'Ridge Rd  & Highland Pl',
 'Upper Sproul Plaza',
 '2495 Bancroft Wy',
 'Foothill-2']

In [ ]:
#combining and parsing finalCrimes
for i in range(len(finalCrimes)):
  for j in range(len(finalCrimes[i])):
    finalCrimes[i][j] = finalCrimes[i][j].strip()
    if(j + 1 == len(finalCrimes[i])):
      finalCrimes[i][j] = finalCrimes[i][j][6:-15]
    else:
      finalCrimes[i][j] = finalCrimes[i][j][2:].strip()
    finalCrimes[i][j] = " ".join(finalCrimes[i][j].split())

In [ ]:
finalCrimes[0]

['22651(n): VC; Posted Tow Away Zone;1 Infraction',
 "487(A): PC; Grand3 Theft:money/Labor/Property; Felony4 10851(A): VC; Take Vehicle W/O Owner's5 Consent/Vehicle Theft; Felony6 OA-BPD: Outside Assist - BPD",
 'DEMO: Demonstration',
 '148.4(A)(1): PC; Tamper/Etc With Fire9 Alarm/Etc; Misdemeanor',
 'Caslty: Injury/Illness']

In [ ]:
(len(finalLocations), len(dates))

(164, 164)

In [ ]:
#for dataset labelling
stackedFinalCrimes = []
for day in finalCrimes:
  for eachCrime in day:
    stackedFinalCrimes.append(eachCrime)
len(stackedFinalCrimes)

In [ ]:
#saving stacked crimes for Co:here
from google.colab import files
df2 = pd.DataFrame()
df2["crimes"] = stackedFinalCrimes
df2.to_csv('sample.csv')
files.download("sample.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#using co:here for predictions
!pip install cohere
import cohere 
from cohere.classify import Example 
model_inputs = df2['crimes']
co = cohere.Client('F6lbJM7XCmlWJNSjwpqE8rdmNjHmF1325dyINMtK')  
response = co.classify(model='2c54e738-3814-4a25-a7bf-3e9489dcd54a-ft', inputs= model_inputs.tolist()) 
response = response.classifications
model_outputs = outputs = [response[i].prediction for i in range(len(x))]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#putting together finalRatings
finalRatings = [[]]
counter3 = 0
for days in finalCrimes:
  cur = []
  for crime in days:
    cur.append(model_outputs[counter3])
    counter3 += 1
  finalRatings.append(cur)
finalRatings = finalRatings[1:]

In [ ]:
#for geomaps labelling
stackedFinalLocations = []
for day in finalLocations:
  for eachLocation in day:
    stackedFinalLocations.append(eachLocation)
len(stackedFinalLocations)

1374

In [ ]:
from google.colab import files
df3 = pd.DataFrame()
df3["Address"] = stackedFinalLocations
df3.to_csv('sample2.csv')
#files.download("sample2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#running the script to do geolocation
#need to manually upload the python_batch_geocode.py file for now
!python3 '/content/python_batch_geocode.py'
newdf = pd.read_csv("/content/output_geocoded.csv")
newdf.head()

In [ ]:
#putting together finalized data for json creation
finalLatitudes = [[]]
finalLongitudes = [[]]
geo_latitudes = newdf['latitude']
geo_longitudes = newdf['longitude']
counter4 = 0
for days in finalLocations:
  curLat = []
  curLong = []
  for locations in days:
    curLat.append(geo_latitudes[counter4])
    curLong.append(geo_longitudes[counter4])
    counter4 += 1
  finalLatitudes.append(curLat)
  finalLongitudes.append(curLong)
finalLatitudes = finalLatitudes[1:]
finalLongitudes = finalLongitudes[1:]

In [ ]:
# making sure variable lengths are the same
print(len(finalCrimes))
print(len(finalLocations))
print(len(finalRatings))
print(len(finalLatitudes))
print(len(finalLongitudes))
###########################
print(len(geo_latitudes)) #location latitude
print(len(geo_longitudes)) #location longitude
print(len(stackedFinalLocations)) #location name
print(len(model_outputs)) # crime ratings
print(len(stackedFinalCrimes)) #crime name

164
164
164
164
164
1374
1374
1374
1374
1374


In [ ]:
#saving file for flask post request
import json 
lst = list()
for i in range(len(stackedFinalCrimes)):
  maps = {"address" : stackedFinalLocations[i], "latitude" : geo_latitudes[i] , "longitude" : geo_longitudes[i], "crime_score": model_outputs[i]}
  lst.append(maps)
print(len(lst))
json_object = json.dumps(lst) 
json_object
text_file = open("/content/jsonfinal.txt", "w")
text_file.write(json_object)
text_file.close()

1374


In [ ]:
#need to rename jsonfinal.txt to jsonfinal.json manually for now
#executes a POST request to CockroachDB
import json
filename = '/content/jsonfinal.json'
with open(filename) as data_file:    
    data = json.load(data_file)
print(data)
import requests
url = 'https://aoauzvp8dk.execute-api.us-west-1.amazonaws.com/default/add'
for i in data:
  x = requests.post(url, json = i)